<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/LabSlope.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
hemo = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Hemodialysis.csv', encoding='latin1')
hemo.shape

(149987, 5)

In [3]:
import pandas as pd
lab = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Lab.csv')
lab.shape

(21641523, 11)

In [4]:
# Create a list from hemo['PatientUID']
patient_uid_list = hemo['PatientUID'].unique().tolist()

In [5]:
# Remove patient UIDs from lab['PatientUID'] that are in the list
lab = lab[~lab['PatientUID'].isin(patient_uid_list)]
lab.shape

(17728574, 11)

In [6]:
lab['PatientVisitUID'].nunique() #มีผู้ป่วย 20967 ที่ไม่ฟอกไต เป็นผู้ป่วยที่เราจะใช้

489786

##**🗂️ Patient Data eGFR age >= 18**

In [7]:
# Define the values to match
result_item_names = ['eGFR(CKD-EPI) age >=18']

# Select rows where 'ResultItemName' matches the specified values
eGFR = lab[lab['ResultItemName'].isin(result_item_names)]
eGFR
labeGFR = eGFR[['LabNo','PatientUID', 'PatientVisitUID', 'ResultItemName', 'ResultValue', 'CWhen']]
labeGFR

,LabNo,PatientUID,PatientVisitUID,ResultItemName,ResultValue,CWhen
248,1,2299,3386255,eGFR(CKD-EPI) age >=18,71.7,2559-08-15 09:51:54.913
255,1,2299,3640924,eGFR(CKD-EPI) age >=18,66.8,2559-11-11 09:40:06.053
271,1,2299,4020970,eGFR(CKD-EPI) age >=18,67.20,2560-03-31 10:18:54.363
286,1,2299,4286612,eGFR(CKD-EPI) age >=18,69.20,2560-07-05 10:40:40.763
331,1,2299,4546471,eGFR(CKD-EPI) age >=18,63.60,2560-10-03 09:19:54.710
...,...,...,...,...,...,...
21628049,159,925649,3617725,eGFR(CKD-EPI) age >=18,77.60,2560-04-17 08:54:35.740
21628297,160,925649,3617725,eGFR(CKD-EPI) age >=18,72.00,2560-04-19 08:41:45.060
21628538,161,925649,3617725,eGFR(CKD-EPI) age >=18,62.90,2560-04-21 08:47:03.167
21628775,162,925649,3617725,eGFR(CKD-EPI) age >=18,59.60,2560-04-24 08:59:53.310


In [8]:
import pandas as pd

# Assuming you have a DataFrame named 'eGFR' with the specified columns
pivot_table = pd.pivot_table(labeGFR, index=['PatientUID', 'PatientVisitUID'], values=['ResultItemName', 'ResultValue', 'CWhen'], aggfunc='first')
pivot_table

CWhen          ResultItemName  \
PatientUID PatientVisitUID                                                    
2299       3386255          2559-08-15 09:51:54.913  eGFR(CKD-EPI) age >=18   
           3640924          2559-11-11 09:40:06.053  eGFR(CKD-EPI) age >=18   
           4020970          2560-03-31 10:18:54.363  eGFR(CKD-EPI) age >=18   
           4286612          2560-07-05 10:40:40.763  eGFR(CKD-EPI) age >=18   
           4546471          2560-10-03 09:19:54.710  eGFR(CKD-EPI) age >=18   
...                                             ...                     ...   
3432056    10312896         2566-01-30 16:16:45.013  eGFR(CKD-EPI) age >=18   
           10408954         2566-02-24 21:05:12.747  eGFR(CKD-EPI) age >=18   
           10573233         2566-04-10 09:51:32.587  eGFR(CKD-EPI) age >=18   
3437526    10191122         2565-12-26 09:29:11.307  eGFR(CKD-EPI) age >=18   
           10541118         2566-03-30 18:44:40.160  eGFR(CKD-EPI) age >=18   

                           ResultValue  
PatientUID PatientVisitUID              
2299       3386255                71.7  
           3640924                66.8  
           4020970               67.20  
           4286612               69.20  
           4546471               63.60  
...                                ...  
3432056    10312896              72.86  
           10408954              68.69  
           10573233              45.08  
3437526    10191122              55.94  
           10541118              56.99  

[318686 rows x 3 columns]

##  **🕑 generate datetime**
    Split CWhen เพราะต้องการ generate year as คศ. Pandas มี outbound
    ให้อยู่ในระยะเวลาของ คศ. แต่ลอง convert column 'CWhen'
    เป็น datetime แล้วทำไม่ได้เลยต้อง split แล้วค่อย merge

In [9]:
import pandas as pd

df = pd.DataFrame(pivot_table)

# Splitting the 'time' column into separate columns
df[['date', 'timestamp']] = df['CWhen'].str.split(' ', expand=True)
df[['year', 'month', 'day']] = df['date'].str.split('-', expand=True)
df[['hour', 'minute', 'second']] = df['timestamp'].str.split(':', expand=True)
df[['second', 'nanosecond']] = df['second'].str.split('.', expand=True)

# Convert year column from BE to AD
df['year'] = df['year'].astype(int) - 543

# Combine columns into a datetime column
df['date'] = pd.to_datetime(df[['year', 'month', 'day',]])
# Convert timestamp to Timedelta
df['timestamp'] = pd.to_timedelta(df['timestamp'])

# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])

# Merge timestamp and date columns
df['datetime'] = df['date'] + df['timestamp']

df_sorted = df.sort_values(by=['PatientUID', 'datetime'])
df_sorted

CWhen          ResultItemName  \
PatientUID PatientVisitUID                                                    
2299       3386255          2559-08-15 09:51:54.913  eGFR(CKD-EPI) age >=18   
           3640924          2559-11-11 09:40:06.053  eGFR(CKD-EPI) age >=18   
           4020970          2560-03-31 10:18:54.363  eGFR(CKD-EPI) age >=18   
           4286612          2560-07-05 10:40:40.763  eGFR(CKD-EPI) age >=18   
           4546471          2560-10-03 09:19:54.710  eGFR(CKD-EPI) age >=18   
...                                             ...                     ...   
3432056    10312896         2566-01-30 16:16:45.013  eGFR(CKD-EPI) age >=18   
           10408954         2566-02-24 21:05:12.747  eGFR(CKD-EPI) age >=18   
           10573233         2566-04-10 09:51:32.587  eGFR(CKD-EPI) age >=18   
3437526    10191122         2565-12-26 09:29:11.307  eGFR(CKD-EPI) age >=18   
           10541118         2566-03-30 18:44:40.160  eGFR(CKD-EPI) age >=18   

                           ResultValue       date              timestamp  \
PatientUID PatientVisitUID                                                 
2299       3386255                71.7 2016-08-15 0 days 09:51:54.913000   
           3640924                66.8 2016-11-11 0 days 09:40:06.053000   
           4020970               67.20 2017-03-31 0 days 10:18:54.363000   
           4286612               69.20 2017-07-05 0 days 10:40:40.763000   
           4546471               63.60 2017-10-03 0 days 09:19:54.710000   
...                                ...        ...                    ...   
3432056    10312896              72.86 2023-01-30 0 days 16:16:45.013000   
           10408954              68.69 2023-02-24 0 days 21:05:12.747000   
           10573233              45.08 2023-04-10 0 days 09:51:32.587000   
3437526    10191122              55.94 2022-12-26 0 days 09:29:11.307000   
           10541118              56.99 2023-03-30 0 days 18:44:40.160000   

                            year month day hour minute second nanosecond  \
PatientUID PatientVisitUID                                                 
2299       3386255          2016    08  15   09     51     54        913   
           3640924          2016    11  11   09     40     06        053   
           4020970          2017    03  31   10     18     54        363   
           4286612          2017    07  05   10     40     40        763   
           4546471          2017    10  03   09     19     54        710   
...                          ...   ...  ..  ...    ...    ...        ...   
3432056    10312896         2023    01  30   16     16     45        013   
           10408954         2023    02  24   21     05     12        747   
           10573233         2023    04  10   09     51     32        587   
3437526    10191122         2022    12  26   09     29     11        307   
           10541118         2023    03  30   18     44     40        160   

                                          datetime  
PatientUID PatientVisitUID                          
2299       3386255         2016-08-15 09:51:54.913  
           3640924         2016-11-11 09:40:06.053  
           4020970         2017-03-31 10:18:54.363  
           4286612         2017-07-05 10:40:40.763  
           4546471         2017-10-03 09:19:54.710  
...                                            ...  
3432056    10312896        2023-01-30 16:16:45.013  
           10408954        2023-02-24 21:05:12.747  
           10573233        2023-04-10 09:51:32.587  
3437526    10191122        2022-12-26 09:29:11.307  
           10541118        2023-03-30 18:44:40.160  

[318686 rows x 13 columns]



> **Lag Generate**

```
 for id in patientID:
    validVisitID = [visit0]
    visitBefore = visit0
    for visit in patientVisit[1:]:
        visitNow = visit
        if timeVisitNow - timeVisitBefore >= 7:
               validVisitID.append(visit)
               visitBefore = visit

```



In [11]:
import pandas as pd

# Step 1: Reset the index to perform calculations
df_sorted_reset = df_sorted.reset_index()

# Step 2: Create lag variables (lag 1 to lag 6) for the 'date' column within each patient
lag_periods = 6
for lag in range(1, lag_periods + 1):
    df_sorted_reset[f'date_lag_{lag}'] = df_sorted_reset.groupby('PatientUID')['date'].shift(lag)

# Step 3: Apply the condition to check for validVisitID within each patient
validVisitID_list = []
for patientID, patient_df in df_sorted_reset.groupby('PatientUID'):

    validVisitID = [patient_df.iloc[0]['PatientVisitUID']]  # Initialize with the first visit
    visitBefore = patient_df.iloc[0]['datetime']

    for _, row in patient_df.iloc[1:].iterrows():
        visitNow = row['datetime']
        if visitNow - visitBefore >= pd.Timedelta(days=7):
            validVisitID.append(row['PatientVisitUID'])
            visitBefore = visitNow

    validVisitID_list.extend(validVisitID)

# Step 4: Filter the DataFrame based on the validVisitID
df_valid_visits = df_sorted_reset[df_sorted_reset['PatientVisitUID'].isin(validVisitID_list)]

# Step 5: Set the index back to ['PatientUID', 'PatientVisitUID']
df_valid_visits.set_index(['PatientUID', 'PatientVisitUID'], inplace=True)

In [12]:
len(set(validVisitID_list))

305172



> **Calculate TimeDiff and Slope**

In [14]:
# Step 6: Generate a new column 'TimeDiff' representing the time difference from 'datetime'
df_valid_visits['TimeDiff'] = df_valid_visits['datetime'].diff().dt.days

#Check If there rows that TimeDiff <7
df_valid_visits[df_valid_visits['TimeDiff'] < 7] ### its the first row of each patientvisitid so it calculate time_diff from another datetime's patient we have to drop all of this

<ipython-input-14-c3bf23bfdf42>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid_visits['TimeDiff'] = df_valid_visits['datetime'].diff().dt.days


,,CWhen,ResultItemName,ResultValue,date,timestamp,year,month,day,hour,minute,second,nanosecond,datetime,date_lag_1,date_lag_2,date_lag_3,date_lag_4,date_lag_5,date_lag_6,TimeDiff
PatientUID,PatientVisitUID,,,,,,,,,,,,,,,,,,,,
2304,3431160,2559-08-30 09:37:00.867,eGFR(CKD-EPI) age >=18,68.9,2016-08-30,0 days 09:37:00.867000,2016,08,30,09,37,00,867,2016-08-30 09:37:00.867,NaT,NaT,NaT,NaT,NaT,NaT,-2257.0
2312,5136829,2561-04-28 14:39:32.587,eGFR(CKD-EPI) age >=18,57.60,2018-04-28,0 days 14:39:32.587000,2018,04,28,14,39,32,587,2018-04-28 14:39:32.587,NaT,NaT,NaT,NaT,NaT,NaT,-1719.0
2354,7003273,2563-01-09 07:57:21.190,eGFR(CKD-EPI) age >=18,56.99,2020-01-09,0 days 07:57:21.190000,2020,01,09,07,57,21,190,2020-01-09 07:57:21.190,NaT,NaT,NaT,NaT,NaT,NaT,-1145.0
2362,3910608,2560-02-20 19:48:50.243,eGFR(CKD-EPI) age >=18,67.60,2017-02-20,0 days 19:48:50.243000,2017,02,20,19,48,50,243,2017-02-20 19:48:50.243,NaT,NaT,NaT,NaT,NaT,NaT,-2229.0
2378,3420850,2559-08-26 09:00:24.393,eGFR(CKD-EPI) age >=18,25.9,2016-08-26,0 days 09:00:24.393000,2016,08,26,09,00,24,393,2016-08-26 09:00:24.393,NaT,NaT,NaT,NaT,NaT,NaT,-2399.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3421578,10151787,2565-12-14 11:30:53.310,eGFR(CKD-EPI) age >=18,47.44,2022-12-14,0 days 11:30:53.310000,2022,12,14,11,30,53,310,2022-12-14 11:30:53.310,NaT,NaT,NaT,NaT,NaT,NaT,-92.0
3426284,10105952,2565-11-29 10:47:37.917,eGFR(CKD-EPI) age >=18,27.88,2022-11-29,0 days 10:47:37.917000,2022,11,29,10,47,37,917,2022-11-29 10:47:37.917,NaT,NaT,NaT,NaT,NaT,NaT,-106.0
3431501,10144649,2565-12-13 13:42:48.510,eGFR(CKD-EPI) age >=18,54.65,2022-12-13,0 days 13:42:48.510000,2022,12,13,13,42,48,510,2022-12-13 13:42:48.510,NaT,NaT,NaT,NaT,NaT,NaT,-87.0


In [15]:

# Convert 'ResultValue' and 'TimeDiff' columns to numeric data types
df_valid_visits['ResultValue'] = pd.to_numeric(df_valid_visits['ResultValue'], errors='coerce')
df_valid_visits['TimeDiff'] = pd.to_numeric(df_valid_visits['TimeDiff'], errors='coerce')

# Drop rows where 'TimeDiff' is NaN, zero, or negative (to avoid division by zero or negative values)
df_valid_visits = df_valid_visits[df_valid_visits['TimeDiff'].notna() & (df_valid_visits['TimeDiff'] > 0)]

# Calculate the slope between 'ResultValue' and 'TimeDiff'
# Note: This is a simple calculation, and you may want to modify it based on your specific requirements
df_valid_visits['Slope'] = df_valid_visits['ResultValue'] / df_valid_visits['TimeDiff']



<ipython-input-15-478c5f2b55b5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid_visits['ResultValue'] = pd.to_numeric(df_valid_visits['ResultValue'], errors='coerce')
<ipython-input-15-478c5f2b55b5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid_visits['TimeDiff'] = pd.to_numeric(df_valid_visits['TimeDiff'], errors='coerce')
<ipython-input-15-478c5f2b55b5>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index



> **CHECK**

```
*   inf row
*   negative time_diff
```




In [16]:
import numpy as np
import pandas as pd

# Assuming you have already calculated 'Slope' as shown in previous steps

# Find rows with infinite 'Slope' values
inf_rows = df_valid_visits[np.isinf(df_valid_visits['Slope'])]
inf_indices = inf_rows.index

# Find rows immediately before the rows with infinite 'Slope' values
rows_before_inf = df_valid_visits[np.isinf(df_valid_visits['Slope'].shift(-1))]

# Concatenate both DataFrames to get rows before and after 'inf_rows'
result = pd.concat([rows_before_inf, inf_rows])

# Print the resulting DataFrame
result

,,CWhen,ResultItemName,ResultValue,date,timestamp,year,month,day,hour,minute,...,nanosecond,datetime,date_lag_1,date_lag_2,date_lag_3,date_lag_4,date_lag_5,date_lag_6,TimeDiff,Slope
PatientUID,PatientVisitUID,,,,,,,,,,,,,,,,,,,,,


In [21]:
df_valid_visits[df_valid_visits['Slope']<=0]


,,CWhen,ResultItemName,ResultValue,date,timestamp,year,month,day,hour,minute,...,nanosecond,datetime,date_lag_1,date_lag_2,date_lag_3,date_lag_4,date_lag_5,date_lag_6,TimeDiff,Slope
PatientUID,PatientVisitUID,,,,,,,,,,,,,,,,,,,,,


In [18]:
df = pd.DataFrame(df_valid_visits)
df

CWhen          ResultItemName  \
PatientUID PatientVisitUID                                                    
2299       3640924          2559-11-11 09:40:06.053  eGFR(CKD-EPI) age >=18   
           4020970          2560-03-31 10:18:54.363  eGFR(CKD-EPI) age >=18   
           4286612          2560-07-05 10:40:40.763  eGFR(CKD-EPI) age >=18   
           4546471          2560-10-03 09:19:54.710  eGFR(CKD-EPI) age >=18   
           4801938          2561-01-04 09:03:28.847  eGFR(CKD-EPI) age >=18   
...                                             ...                     ...   
3432056    10235244         2566-01-09 12:31:53.220  eGFR(CKD-EPI) age >=18   
           10306813         2566-01-27 12:22:43.000  eGFR(CKD-EPI) age >=18   
           10408954         2566-02-24 21:05:12.747  eGFR(CKD-EPI) age >=18   
           10573233         2566-04-10 09:51:32.587  eGFR(CKD-EPI) age >=18   
3437526    10541118         2566-03-30 18:44:40.160  eGFR(CKD-EPI) age >=18   

                            ResultValue       date              timestamp  \
PatientUID PatientVisitUID                                                  
2299       3640924                66.80 2016-11-11 0 days 09:40:06.053000   
           4020970                67.20 2017-03-31 0 days 10:18:54.363000   
           4286612                69.20 2017-07-05 0 days 10:40:40.763000   
           4546471                63.60 2017-10-03 0 days 09:19:54.710000   
           4801938                58.40 2018-01-04 0 days 09:03:28.847000   
...                                 ...        ...                    ...   
3432056    10235244               48.51 2023-01-09 0 days 12:31:53.220000   
           10306813               67.71 2023-01-27        0 days 12:22:43   
           10408954               68.69 2023-02-24 0 days 21:05:12.747000   
           10573233               45.08 2023-04-10 0 days 09:51:32.587000   
3437526    10541118               56.99 2023-03-30 0 days 18:44:40.160000   

                            year month day hour minute  ... nanosecond  \
PatientUID PatientVisitUID                              ...              
2299       3640924          2016    11  11   09     40  ...        053   
           4020970          2017    03  31   10     18  ...        363   
           4286612          2017    07  05   10     40  ...        763   
           4546471          2017    10  03   09     19  ...        710   
           4801938          2018    01  04   09     03  ...        847   
...                          ...   ...  ..  ...    ...  ...        ...   
3432056    10235244         2023    01  09   12     31  ...        220   
           10306813         2023    01  27   12     22  ...        000   
           10408954         2023    02  24   21     05  ...        747   
           10573233         2023    04  10   09     51  ...        587   
3437526    10541118         2023    03  30   18     44  ...        160   

                                          datetime date_lag_1 date_lag_2  \
PatientUID PatientVisitUID                                                 
2299       3640924         2016-11-11 09:40:06.053 2016-08-15        NaT   
           4020970         2017-03-31 10:18:54.363 2016-11-11 2016-08-15   
           4286612         2017-07-05 10:40:40.763 2017-03-31 2016-11-11   
           4546471         2017-10-03 09:19:54.710 2017-07-05 2017-03-31   
           4801938         2018-01-04 09:03:28.847 2017-10-03 2017-07-05   
...                                            ...        ...        ...   
3432056    10235244        2023-01-09 12:31:53.220 2022-12-15        NaT   
           10306813        2023-01-27 12:22:43.000 2023-01-09 2022-12-15   
           10408954        2023-02-24 21:05:12.747 2023-01-30 2023-01-27   
           10573233        2023-04-10 09:51:32.587 2023-02-24 2023-01-30   
3437526    10541118        2023-03-30 18:44:40.160 2022-12-26        NaT   

                           date_lag_3 date_lag_4 date_lag_5 date

In [19]:
df = df[['ResultValue','datetime','TimeDiff', 'Slope']]
df

ResultValue                datetime  TimeDiff  \
PatientUID PatientVisitUID                                                  
2299       3640924                66.80 2016-11-11 09:40:06.053      87.0   
           4020970                67.20 2017-03-31 10:18:54.363     140.0   
           4286612                69.20 2017-07-05 10:40:40.763      96.0   
           4546471                63.60 2017-10-03 09:19:54.710      89.0   
           4801938                58.40 2018-01-04 09:03:28.847      92.0   
...                                 ...                     ...       ...   
3432056    10235244               48.51 2023-01-09 12:31:53.220      25.0   
           10306813               67.71 2023-01-27 12:22:43.000      17.0   
           10408954               68.69 2023-02-24 21:05:12.747      28.0   
           10573233               45.08 2023-04-10 09:51:32.587      44.0   
3437526    10541118               56.99 2023-03-30 18:44:40.160      94.0   

                               Slope  
PatientUID PatientVisitUID            
2299       3640924          0.767816  
           4020970          0.480000  
           4286612          0.720833  
           4546471          0.714607  
           4801938          0.634783  
...                              ...  
3432056    10235244         1.940400  
           10306813         3.982941  
           10408954         2.453214  
           10573233         1.024545  
3437526    10541118         0.606277  

[285181 rows x 4 columns]


> **Cut data**
```
เช็คว่าแต่ละคนมีจำนวนสโลปที่คำนวณจากระยะเวลาห่างกันอย่างน้อย 3 เดือน
อย่างน้อย 2 สโลปไหม ถ้ามีไม่ครบให้ตัดคนนั้นทิ้ง
```


In [22]:
import pandas as pd

# Assuming you have a DataFrame 'eGFR_Patient' with index 'PatientUID' and columns 'datetime', 'slopes', and 'time_difference'

# Create an empty list to store the valid PatientUIDs
valid_patient_uids = []

# Group the DataFrame by index 'PatientUID'
grouped = df.groupby(level='PatientUID')

# Iterate over each group
for patient_uid, group in grouped:
    # Check if the group has at least 2 rows
    if len(group) >= 3:
        # Calculate the minimum and maximum datetime in the group
        min_datetime = group['datetime'].min()
        max_datetime = group['datetime'].max()

        # Calculate the time difference in days between the minimum and maximum datetime
        time_diff = (max_datetime - min_datetime).days

        # Check if the time difference is at least 90 days (3 months)
        if time_diff >= 90:
            valid_patient_uids.append(patient_uid)

# Filter the original DataFrame based on the valid PatientUIDs
filtered_df = df.loc[valid_patient_uids]

In [26]:
filtered_df[filtered_df['Slope']<=0]

,,ResultValue,datetime,TimeDiff,Slope
PatientUID,PatientVisitUID,,,,


In [27]:
filtered_df[filtered_df['Slope']==float('inf')]

,,ResultValue,datetime,TimeDiff,Slope
PatientUID,PatientVisitUID,,,,


In [29]:
filtered_df.reset_index()['PatientUID'].nunique()

17711

In [30]:
a = filtered_df.reset_index()

In [31]:
a.to_excel('LabSlope.xlsx', index=False)